In [ ]:
# ╔═══════════════════════════════════════════════╗
#   Build & save  phase_tests_v3.ipynb  → /content
#   • zero external deps     • auto‑visible file
# ╚═══════════════════════════════════════════════╝
import nbformat as nbf, textwrap, mpmath as mp, numpy as np, math, cmath, pandas as pd
mp.mp.dps = 120  # precision

def nth_prime(k):
    if k == 1: return 2
    primes=[2]; n=3
    while len(primes) < k:
        if all(n % p for p in primes): primes.append(n)
        n += 2
    return primes[-1]

# ── notebook cells ───────────────────────────────
nb = nbf.v4.new_notebook(); cells=[]
cells.append(nbf.v4.new_markdown_cell("# Fibonacci → Phase Tests v3\nAuto‑generated, zero‑input notebook."))

cells.append(nbf.v4.new_code_cell("import mpmath as mp, numpy as np, pandas as pd, math, cmath\nmp.mp.dps=120"))

cells.append(nbf.v4.new_code_cell(textwrap.dedent("""
# Fibonacci helpers
F=[0,1]
for _ in range(2,35): F.append(F[-1]+F[-2])
def fib_bits(n, rev=False):
    a,b='0','01'; w='0'
    while len(w)<n: w=''.join(b if c=='0' else a for c in w)
    s=w[:n]; return s[::-1] if rev else s
frac = lambda bits: mp.mpf(int(bits,2))/(1<<len(bits))
def B(n, rev=False): return frac(fib_bits(F[n], rev))
def tail(n,L=512):   return frac(fib_bits(F[n]+L)[F[n]:F[n]+L])
def prime_pref(k):   return frac(fib_bits(nth_prime(k)))
""")))

cells.append(nbf.v4.new_code_cell(textwrap.dedent("""
# Constants
alpha_inv=mp.mpf('137.035999146')
C={'α⁻¹/1000':1/(1000*alpha_inv),'π':mp.pi,'e':mp.e,'√2':mp.sqrt(2),
   'Z_c':mp.mpf('0.27'),'T_λ':mp.mpf('2.17'),'T_NI':mp.mpf('1.2')}
""")))

cells.append(nbf.v4.new_code_cell(textwrap.dedent("""
# Transforms
odds=lambda x:x/(1-x); comp=lambda x:1-x; recip=lambda x:1/x
geom={'raw':lambda x:x,'π/4':lambda x:x*mp.pi/4,'√2/3':lambda x:x*mp.sqrt(2)/3}
exps=[-1/9,-1/27,-1/81,-1/729,1/9,1/27]
def holos(x):
    z=cmath.exp(1j*math.pi*float(x))
    return {'e^{iπB}_Re':z.real,'e^{iπB}_Im':z.imag,'e^{iπB}_Abs':abs(z)}
def diff_ratio(n): return (B(n+1)-B(n))/(B(n)-B(n-1))
def log_slope(n):  return mp.log(abs(B(n+1)-B(n)))/mp.log(abs(B(n)-B(n-1)))
""")))

cells.append(nbf.v4.new_code_cell(textwrap.dedent("""
# Analysis loop
rows=[]
for n in range(15,25):
    base={'B':B(n),'B_rev':B(n,True),'Tail':tail(n),
          'Prime':prime_pref(n-14),'Odds':odds(B(n)),
          'Comp':comp(B(n)),'Recip':recip(B(n)),
          'Diff':diff_ratio(n),'Slope':log_slope(n)}
    base.update(holos(B(n)))
    for gname,gfun in geom.items():
        for p in exps: base[f'{gname}^{p}']=gfun(B(n))**p
    for t,v in base.items():
        for k,c in C.items():
            ppm=abs(v-c)/c*1e6 if math.isfinite(float(v)) else math.inf
            rows.append((n,F[n],t,k,float(v),ppm))
df=pd.DataFrame(rows,columns=['n','F_n','transform','constant','value','ppm'])
def flag(a):
    if np.isinf(a).any(): return '∞'
    s=np.diff(np.log10(a));
    if (s<0).all(): return '↓'
    if (s>0).all(): return '↑'
    for k in range(2,6):
        if np.allclose(a[:-k],a[k:]): return f'↻{k}'
    return 'osc'
behav={key:flag(g.sort_values('n')['ppm'].values)
       for key,g in df.groupby(['transform','constant'])}
df['behaviour']=df.set_index(['transform','constant']).index.map(behav)
display(df[df.behaviour!='↑'].head(40))
print("DataFrame 'df' ready; try df[df.behaviour=='↓']")
""")))

nb['cells']=cells

# ── save notebook to /content (visible in sidebar) ───────────
path='/content/phase_tests_v3.ipynb'
with open(path,'w',encoding='utf-8') as f: nbf.write(nb,f)
print("✅ Notebook saved →", path)
